In [1]:
import psycopg

from psycopg.rows import dict_row
from fishsense_api_workflow_worker.database import Database
from fishsense_api_workflow_worker.models.camera import Camera
from fishsense_api_workflow_worker.models.dive import Dive
from fishsense_api_workflow_worker.models.image import Image

In [2]:
PG_CONN_STR = (
    f"postgres://postgres:aoiwhfe;ajaef@"
    f"fabricant-prod.ucsd.edu:5432/"
    f"postgres"
)

In [3]:
SELECT_CAMERAS_QUERY = "SELECT serial_number, name FROM cameras"

In [4]:
database = Database()
await database.init_database()

In [5]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    rows = cur.execute(query=SELECT_CAMERAS_QUERY)

    for row in rows:
        existing_camera = await database.select_camera_by_serial_number(row["serial_number"])
        if existing_camera is not None:
            print(existing_camera)
            continue

        camera = Camera(serial_number=row["serial_number"], name=row["name"])
        await database.insert_camera(camera)

id=1 name='FSL-01' serial_number='BJ6C69506'
id=2 name='FSL-02' serial_number='BJ6C83748'
id=3 name='FSL-03' serial_number='BJ6C85528'
id=4 name='FSL-04' serial_number='BJ6C85524'
id=5 name='FSL-05' serial_number='BJ6C67988'
id=6 name='FSL-06' serial_number='BJ6C67987'
id=7 name='FSL-08' serial_number=''
id=8 name='FSL-09' serial_number='BJPA75620'
id=9 name='FSL-11' serial_number='BJPA75624'
id=10 name='FSL-07' serial_number='BJ6C67989'
id=11 name='FSL-10' serial_number='BJPA75627'


In [6]:
SELECT_DIVES_QUERY = "SELECT path, date FROM dives ORDER BY date"
SELECT_IMAGES_QUERY = "SELECT path, dive, date, image_md5, camera_sn FROM images ORDER BY date"

In [7]:
dives = []
images = []

with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    dive_rows = cur.execute(query=SELECT_DIVES_QUERY)
    for dive_row in dive_rows:
        if ".Trashes" in dive_row["path"]:
            continue

        dives.append(dive_row)

    image_rows = cur.execute(query=SELECT_IMAGES_QUERY)
    for image_row in image_rows:
        images.append(image_row)

In [8]:
for dive in dives:
    existing_dive = await database.select_dive_by_path(dive["path"])
    if existing_dive is not None:
        print(existing_dive)
        continue

    dive_images = [i for i in images if i["dive"] == dive["path"]]
    image_dates = [i["date"] for i in dive_images]

    image_dates.sort()
    end_date = image_dates[-1]

    camera = await database.select_camera_by_serial_number(images[-1]["camera_sn"])

    dive = Dive(path=dive["path"], dive_datetime=end_date, camera_id=camera.id)
    await database.insert_dive(dive)

camera_id=4 path='2023-09-07 REEF Data Dump/080123_FSL-01 Photos' id=1 dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27)
camera_id=4 path='2024.06.20.REEF/08_2023/080123_FSL-01 Photos' id=2 dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27)
camera_id=4 path='2025-02-10 REEF Data Dump SMILE 6/080123_FSL-01 Photos' id=3 dive_datetime=datetime.datetime(2023, 8, 1, 12, 46, 27)
camera_id=4 path='2023.08.03.FishSense.FSL-01D/Hogfish02_MolHITW_0943_080323' id=4 dive_datetime=datetime.datetime(2023, 8, 3, 9, 43, 11)
camera_id=4 path='2023.08.03.FishSense.FSL-01D/Hogfish01_MolHITW_0926_080323' id=5 dive_datetime=datetime.datetime(2023, 8, 3, 9, 27, 23)
camera_id=4 path='2023.08.03.FishSense.FSL-01D/Hogfish01_MolPeLe_1029_080323' id=6 dive_datetime=datetime.datetime(2023, 8, 3, 10, 29, 49)
camera_id=4 path='2023.08.03.FishSense.FSL-01D/BlueParrot01_MolHITW_0924_080323' id=7 dive_datetime=datetime.datetime(2023, 8, 3, 9, 24, 36)
camera_id=4 path='2023.08.03.FishSense.FSL-01D/DogSnapper

In [9]:
dives = await database.select_dives()

for dive in dives:
    images_per_dive = [i for i in images if i["dive"] == dive.path]
    images_per_dive.sort(key=lambda i: i["date"])

    for image_row in images_per_dive:
        existing_image = await database.select_image_by_path(image_row["path"])

        if existing_image is not None:
            continue

        existing_checksum = await database.select_image_by_checksum(image_row["image_md5"])

        camera = await database.select_camera_by_serial_number(image_row["camera_sn"])
        image = Image(path=image_row["path"], taken_datetime=image_row["date"], checksum=image_row["image_md5"], dive_id=dive.id, camera_id=camera.id, is_canonical=existing_checksum is None)
        await database.insert_image(image)